# Use Relaxed VFE to Predict Fractional Vacancy Occupancy
<b> nanoHUB tools by: </b>  <i>Mackinzie S. Farnell, Zachary D. McClure</i> and <i>Alejandro Strachan</i>, Materials Engineering, Purdue University <br>

We predict relaxed vacancy formation energy of equiatomic CrFeCoNi and calculate the vacancy occupancy. We calculate vacancy occupancy at a range of temperatures and plot distribution of occupancy values. 

Outline
1. Predict Relaxed VFE Using Bispectrum Coefficients
2. Calculate Fractional Occupancy and Plot Distribution
3. Plot Vacancy Concentration versus Inverse Temperature

In [ ]:
# import libraries we need
import json as js
import numpy as np
import math as m
import plotly.offline as p
import plotly.graph_objs as go
from keras.models import load_model
import os.path
from os import path
import pymatgen as pymat
import csv

## 1. Predict Relaxed VFE Using Bispectrum Coefficients
We use bispectrum coefficients as model inputs to predict relaxed vfe using a model trained on equiatomic CrFeCoNi. We use the bispectrum coefficients generated from running the [Build Structure and Calculate Bispectrum Coefficients:](generate_struct_BS_coeffs.ipynb) notebook. If you have not run this notebook, we use a default file that has the bispectrum coefficients for a structure of equiatomic CrFeCoNi.

In [ ]:
filename = 'bs_coeffs.json'

if not path.exists(filename):
    filename = '../data/CrFeCoNi.json'

with open(filename, 'r') as f:
    bs_data = js.load(f)

# get list of elements and bispectrum coefficients from data variable
elements = bs_data['element']
bs_coeffs = bs_data['Unrelaxed_Bispectrum_Coefficients']

bs_coeffs_array = np.array([])
elements_array = np.array([])

# iterate through elements and get input and output properties for desired element
for i, val in enumerate(elements):
    bs_coeffs_array = np.append(bs_coeffs_array, np.asarray(bs_coeffs[i]))
    if (val == 'Cr'):
        elements_array = np.append(elements_array, 24)
    elif (val == 'Fe'):
        elements_array = np.append(elements_array, 26)
    elif (val == 'Co'):
        elements_array = np.append(elements_array, 27)
    elif (val == 'Ni'):
        elements_array = np.append(elements_array, 28)
    elif (val == 'Cu'):
        elements_array = np.append(elements_array, 29)

num_rows = int (bs_coeffs_array.shape[0]/55)
bs_coeffs_array = np.reshape(bs_coeffs_array, (num_rows, 55))
bs_coeffs_array = bs_coeffs_array.astype(np.float)

inputs = bs_coeffs_array

# element number is included as input to model
elements_array = elements_array[np.newaxis].T
inputs = np.append(inputs, elements_array, 1)

# now we will add the central atom descriptors

# declare function to query property from pymatgen for a given element
def get_property(element, property):
    element_object = pymat.Element(element)
    element_prop = getattr(element_object, property)
    return element_prop

# list of properties to add central atom descriptors for
properties = ['atomic_radius_calculated', 'atomic_radius', 'atomic_mass', 
              'poissons_ratio', 'electrical_resistivity', 'thermal_conductivity', 
              'brinell_hardness']
   
# iterate through all properties to add
for add_property in properties:
    atom_properties = []
    # determine which element to get property for
    for i in elements:
        prop = get_property(i, add_property)
        atom_properties.append(float (prop))

    # add property to array of inputs
    atom_properties = np.asarray(atom_properties) 
    atom_properties = atom_properties[np.newaxis].T
    inputs = np.append(inputs, atom_properties, 1)

# normalize bispectrum coefficients using mean and standard deviation values for equiatomic CoCrFeNi
# we use mean and st dev from this file because we use the equiatomic CrFeCoNi model to make prediction
stats_file_name = "my_models/63/relaxed_vfe_stats_CrFeCoNi.json"
# load the stats dictionary
f = open(stats_file_name,"r")
stats_data = js.load(f)
f.close()
  
for num in stats_data:
    stats_data[num] = np.asarray(stats_data[num])

dims = bs_coeffs_array.shape
for j in range(0, dims[0]):
    inputs[j] = ((inputs[j] - stats_data["means_ins"])/stats_data["stdevs_ins"])
inputs = np.nan_to_num(inputs)

#load model trained on equiatomic CoCrFeNi
model = load_model("my_models/63/relaxed_vfe_model_CrFeCoNi.h5")
relaxed_vfe_predict = model.predict(inputs)

# undo the normalization of the relaxed-vfe so that we have the correct values
relaxed_vfe_NN = (relaxed_vfe_predict * stats_data["stdevs_outs"] + stats_data["means_outs"])

relaxed_vfe = relaxed_vfe_NN

# array to store relaxed vfe for each atom in the sample
output_properties_array_Cr = np.array([])
output_properties_array_Fe = np.array([])
output_properties_array_Co = np.array([])
output_properties_array_Ni = np.array([])
output_properties_array_Cu = np.array([])

num_Cr = 0
num_Fe = 0
num_Co = 0
num_Ni = 0
num_Cu = 0

# iterate through elements and get relaxed_vfe for desired element
for i, val in enumerate(elements_array):
    if (val == 24):
        output_properties_array_Cr = np.append(output_properties_array_Cr, relaxed_vfe[i])
        num_Cr = num_Cr + 1
    elif (val == 26):
        output_properties_array_Fe = np.append(output_properties_array_Fe, relaxed_vfe[i])
        num_Fe = num_Fe + 1
    elif (val == 27):
        output_properties_array_Co = np.append(output_properties_array_Co, relaxed_vfe[i])
        num_Co = num_Co + 1
    elif (val == 28):
        output_properties_array_Ni = np.append(output_properties_array_Ni, relaxed_vfe[i])
        num_Ni = num_Ni + 1
    elif (val == 29):
        output_properties_array_Cu = np.append(output_properties_array_Cu, relaxed_vfe[i])
        num_Cu = num_Cu + 1

## 2. Calculate Fractional Occupancy and Plot Distribution
We use relaxed vfe predictions to calculate fractional occupancy using the equation below. The calculation is performed at multiple temperatures and distributions for the fractional occupancy are shown. 

$$ \frac{N_0}{N} = \exp(-\frac{Q_v}{kT})  $$

In [ ]:
vac_concentration_Cr = []
vac_concentration_Fe = []
vac_concentration_Co = []
vac_concentration_Ni = []
vac_concentration_Cu = []

# temperatures in K
temps = [500, 750, 1000, 1250, 1500, 1750, 2000, 2250, 2500] 

# boltzmann constant in eV/K
k = 0.0000862 

# iterate through each temperature
for t in temps:
    # array to store vacancy fraction
    vacancy_fraction_Cr = np.array([])
    vacancy_fraction_Fe = np.array([])
    vacancy_fraction_Co = np.array([])
    vacancy_fraction_Ni = np.array([])
    vacancy_fraction_Cu = np.array([])
    Cr = 0
    Fe = 0
    Co = 0
    Ni = 0
    Cu = 0
    # iterate over all the relaxed_vfe values
    for value in output_properties_array_Cr:
        # calculate vacancy fraction
        vacancy_fraction_i_Cr = m.exp(-value/(k*t))
        Cr = Cr + m.exp(-value/(k*t))
        # append to vacancy fraction array
        vacancy_fraction_Cr = np.append(vacancy_fraction_Cr, vacancy_fraction_i_Cr)
    for value in output_properties_array_Fe:
        # calculate vacancy fraction
        vacancy_fraction_i_Fe = m.exp(-value/(k*t))
        Fe = Fe + m.exp(-value/(k*t))
        # append to vacancy fraction array
        vacancy_fraction_Fe = np.append(vacancy_fraction_Fe, vacancy_fraction_i_Fe)
    for value in output_properties_array_Co:
        # calculate vacancy fraction
        vacancy_fraction_i_Co = m.exp(-value/(k*t))
        Co = Co + m.exp(-value/(k*t))
        # append to vacancy fraction array
        vacancy_fraction_Co = np.append(vacancy_fraction_Co, vacancy_fraction_i_Co)
    for value in output_properties_array_Ni:
        # calculate vacancy fraction
        vacancy_fraction_i_Ni = m.exp(-value/(k*t))
        Ni = Ni + m.exp(-value/(k*t))
        # append to vacancy fraction array
        vacancy_fraction_Ni = np.append(vacancy_fraction_Ni, vacancy_fraction_i_Ni)
    for value in output_properties_array_Cu:
        # calculate vacancy fraction
        vacancy_fraction_i_Cu = m.exp(-value/(k*t))
        Cu = Cu + m.exp(-value/(k*t))
        # append to vacancy fraction array
        vacancy_fraction_Cu = np.append(vacancy_fraction_Cu, vacancy_fraction_i_Cu)
                                        
    # Store vac. concentration
    vac_concentration_Cr = np.append(vac_concentration_Cr, Cr/num_Cr)
    vac_concentration_Fe = np.append(vac_concentration_Fe, Fe/num_Fe) 
    vac_concentration_Co = np.append(vac_concentration_Co, Co/num_Co)
    vac_concentration_Ni = np.append(vac_concentration_Ni, Ni/num_Ni)
    #vac_concentration_Cu = np.append(vac_concentration_Cu, Cu/num_Cu)
    # plot histogram of vacancy_fraction
    fig = go.Figure()
    fig.add_trace(go.Histogram(x=vacancy_fraction_Cr, name = 'Cr', nbinsx = 100, marker_color = 'red'))
    fig.add_trace(go.Histogram(x=vacancy_fraction_Fe, name = 'Fe', nbinsx = 100, marker_color = 'orange'))
    fig.add_trace(go.Histogram(x=vacancy_fraction_Co, name = 'Co', nbinsx = 125, marker_color = 'blue'))
    fig.add_trace(go.Histogram(x=vacancy_fraction_Ni, name = 'Ni', nbinsx = 100, marker_color = 'green'))
    #fig.add_trace(go.Histogram(x=vacancy_fraction_Cu, name = 'Cu', nbinsx = 100, marker_color = 'grey'))
    fig.update_layout(barmode='overlay')
    fig.update_traces(opacity=0.75)
    fig.update_layout(
        xaxis_title="Fractional Occupancy - {} K".format(t),
        yaxis_title="Frequency",
        font=dict(
            family="Times New Roman, monospace",
            size=24,
            color= "black"
        )
    )
    fig.show()
    

## 3. Plot Vacancy Concentration versus Inverse Temperature
Here, we plot the vacancy concentration versus the inverse temperature. On the plot we show the values for the mean and the values for the distribution. Initially, the mean value is near the distribution, but as the x-values rise, the mean values begin to diverge from the distribution.

In [ ]:
inv_temp = []
mean_VFE_Cr = []
mean_VFE_Fe = []
mean_VFE_Co = []
mean_VFE_Ni = []
mean_VFE_Cu = []
for t in temps:
    inv_temp = np.append(inv_temp, 1000/t)
    mean_VFE_Cr = np.append(mean_VFE_Cr, m.exp(-1.0*np.mean(output_properties_array_Cr)/(k*t)))
    mean_VFE_Fe = np.append(mean_VFE_Fe, m.exp(-1.0*np.mean(output_properties_array_Fe)/(k*t)))
    mean_VFE_Co = np.append(mean_VFE_Co, m.exp(-1.0*np.mean(output_properties_array_Co)/(k*t)))
    mean_VFE_Ni = np.append(mean_VFE_Ni, m.exp(-1.0*np.mean(output_properties_array_Ni)/(k*t)))
    mean_VFE_Cu = np.append(mean_VFE_Cu, m.exp(-1.0*np.mean(output_properties_array_Cu)/(k*t)))

fig = go.Figure()
fig.add_trace(go.Scatter(x=inv_temp, y=vac_concentration_Cr,
              mode='markers',
              name='Dist. Cr',
              marker= dict(size= 10, 
                           color= 'red', 
                           opacity=0.5,
                           line=dict(width=2, color='Black')
                           )
             ))
fig.add_trace(go.Scatter(x=inv_temp, y=vac_concentration_Fe,
              mode='markers',
              name='Dist. Fe',
              marker= dict(size= 10, 
                           color= 'orange', 
                           opacity=0.5,
                           line=dict(width=2, color='Black')
                           )
             ))
fig.add_trace(go.Scatter(x=inv_temp, y=vac_concentration_Co,
              mode='markers',
              name='Dist. Co',
              marker= dict(size= 10, 
                           color= 'blue', 
                           opacity=0.5,
                           line=dict(width=2, color='Black')
                           )
             ))
fig.add_trace(go.Scatter(x=inv_temp, y=vac_concentration_Ni,
            mode='markers',
            name='Dist. Ni',
            marker= dict(size= 10, 
                         color= 'green', 
                         opacity=0.5,
                         line=dict(width=2, color='Black')
                         )
            ))
# uncomment these lines if your system includes Cu
#fig.add_trace(go.Scatter(x=inv_temp, y=vac_concentration_Cu,
#              mode='markers',
#              name='Dist. Cu',
#              marker= dict(size= 10, 
#                           color= 'grey', 
#                           opacity=0.5,
#                           line=dict(width=2, color='Black')
#                           )
#             ))

fig.add_trace(go.Scatter(x=inv_temp, y=mean_VFE_Cr,
            mode='lines',
            name='Mean Cr',
            marker=dict(size=12, color="red")))
fig.add_trace(go.Scatter(x=inv_temp, y=mean_VFE_Fe,
            mode='lines',
            name='Mean Fe',
            marker=dict(size=12, color="orange")))
fig.add_trace(go.Scatter(x=inv_temp, y=mean_VFE_Co,
            mode='lines',
            name='Mean Co',
            marker=dict(size=12, color="blue")))
fig.add_trace(go.Scatter(x=inv_temp, y=mean_VFE_Ni,
            mode='lines',
            name='Mean Ni',
            marker=dict(size=12, color="green")))
# uncomment these lines if your system includes Cu
#fig.add_trace(go.Scatter(x=inv_temp, y=mean_VFE_Cu,
#            mode='lines',
#            name='Mean Cu',
#            marker=dict(size=12, color="grey")))

# plot figure
fig.update_yaxes(type="log")
fig.update_layout(
    showlegend=True,
    xaxis_title="1000/T [1/K]",
    yaxis_title=r"$\Huge{x_{v}}$",
    title = "Vacancy Fraction",
    height=800,
    width=800,
    font=dict(
        family="Times New Roman, monospace",
        size=24,
        color= "black"
    )    
)
fig.show()

In [ ]:
print('done!')